# SakepediaのAPIをクローリングしてjsonファイルを作成する

## 銘柄API
銘柄は一括では取得できないので県別jsonとして作成する。
県別jsonの一覧を`sakes.json`として作成する。

### ファイル名
* bottles.json
* bottles/{県名英字}.json

### 項目
銘柄は以下が取得可能(※ない場合は空白文字)
* _id
* url
* subname
* aminoAcidContent
* alcoholContent
* price
* award
* sakeYeast
* brand
* volume
* sakeRiceExceptForKojiMaking
* sakeMeterValue
* mariage
* type
* ricePolishingRate
* brewery
* breweryYear
* matchDrinkingVessel
* matchDrinkingSceneAndTarget
* matchDrinkingTemperature
* prefecture
* starterCulture
* acidity
* description
* riceForMakingKoji


## 酒蔵API
### ファイル名
* breweries.json

### 項目
酒蔵は以下が取得可能(※ない場合は空白文字)
* 蔵元
* カナ
* 都道府県
* 住所
* email
* tel
* url
* 創業

jsonファイルは全てUTF8でインデントは半角スペース*2です。


In [47]:
#定数
DRIVE_ROOT_DIR="./drive/My Drive/Code for Sake/json/"
API_URL = 'https://www.sakepedia.io/api/'
PREFUCTURE_LIST = {
                   '北海道':'hokkaido.json',
                   '青森県':'aomori.json',
                   '岩手県':'iwate.json',
                   '宮城県':'miyagi.json',
                   '秋田県':'akita.json',
                   '山形県':'yamagata.json',
                   '福島県':'fukushima.json',
                   '茨城県':'ibaraki.json',
                   '栃木県':'tochigi.json',
                   '群馬県':'gunma.json',
                   '埼玉県':'saitama.json',
                   '千葉県':'chiba.json',
                   '東京都':'tokyo.json',
                   '神奈川県':'kanagawa.json',
                   '新潟県':'niigata.json',
                   '富山県':'toyama.json',
                   '石川県':'ishikawa.json',
                   '福井県':'fukui.json',
                   '山梨県':'yamanashi.json',
                   '長野県':'nagano.json',
                   '岐阜県':'gifu.json',
                   '静岡県':'shizuoka.json',
                   '愛知県':'aichi.json',
                   '三重県':'mie.json',
                   '滋賀県':'shiga.json',
                   '京都府':'kyoto.json',
                   '大阪府':'osaka.json',
                   '兵庫県':'hyogo.json',
                   '奈良県':'nara.json',
                   '和歌山県':'wakayama.json',
                   '鳥取県':'tottori.json',
                   '島根県':'shimane.json',
                   '岡山県':'okayama.json',
                   '広島県':'hiroshima.json',
                   '山口県':'yamaguchi.json',
                   '徳島県':'tokushima.json',
                   '香川県':'kagawa.json',
                   '愛媛県':'ehime.json',
                   '高知県':'kochi.json',
                   '福岡県':'fukuoka.json',
                   '佐賀県':'saga.json',
                   '長崎県':'nagasaki.json',
                   '熊本県':'kumamoto.json',
                   '大分県':'oita.json',
                   '宮崎県':'miyazaki.json',
                   '鹿児島県':'kagoshima.json',
                   '沖縄県':'okinawa.json',
}

In [51]:
import json
import urllib.request
import urllib.parse
import os

#jsonを読み込む
def read_json(url):
    res = urllib.request.urlopen(url)
    res=res.read().decode('utf8')
    data = json.loads(res)
    return data

#jsonとして保存する
def save_json(path,data):
    json_str = json.dumps(data, indent=2)
    file_path = os.path.dirname(path)
    if not os.path.exists(file_path):
        os.makedirs(file_path)
    with open(path, mode='w') as f:
        f.write(json_str.encode().decode('unicode-escape'))


In [49]:
#県別の銘柄APIをjsonとして保存
def api2jsonfile(prefecture, json_name):
    data = read_json(API_URL+'sakes?prefecture='+urllib.parse.quote(prefecture))
    # 不要なデータを削除
    for item in data:
      if 'reviews' in item:
        del item['reviews']
      if 'userid' in item:
        del item['userid']
      if 'username' in item:
        del item['username']      
    save_json(DRIVE_ROOT_DIR+json_name,data)

#銘柄json一覧を保存
def create_sakes_indexfile(json_name):
  save_json(DRIVE_ROOT_DIR+json_name,PREFUCTURE_LIST)

#酒蔵APIをjsonとして保存
def breweries_api2jsonfile(json_name):
  #酒蔵情報
  data = read_json(API_URL+'breweries')

  json = []
  header = None;
  for line in data:
    if (header is None):
      #ヘッダーを保存
      header = line
      continue
    else:
      #ヘッダーを項目名としてdictに変換
      obj = {};
      for index in range(len(line)):
        obj[header[index]] = line[index]
      json.append(obj)

  save_json(DRIVE_ROOT_DIR+json_name, json)

In [52]:
#銘柄API
for k,v in PREFUCTURE_LIST.items():
  api2jsonfile(k,'bottles/'+v)

create_sakes_indexfile('bottles.json')

In [53]:
#酒蔵API
breweries_api2jsonfile('breweries.json')